<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [65]:
import folium
import wget
import pandas as pd

In [66]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [67]:
# Download and read the `spacex_launch_geo.csv`
#spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_csv_file = 'spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)

In [68]:
print (spacex_df.columns)

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long'],
      dtype='object')


Now, you can take a look at what are the coordinates for each site.


In [69]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [74]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [80]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [81]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    circle = folium.Circle(coordinate, radius=5000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site']))
    site_map.add_child(circle)

    # Create a marker for each launch site
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    site_map.add_child(marker)

# Save the map as an HTML file
site_map.save('launch_sites_map.html')

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


1. Are all launch sites in proximity to the Equator line?

No, not all launch sites are in proximity to the Equator line. While some launch sites, such as those in Florida (USA) and Guiana Space Centre (French Guiana), are relatively close to the Equator, others, like Vandenberg Space Force Base (California, USA) and Plesetsk Cosmodrome (Russia), are located at higher latitudes.
This makes sense, as launch sites can be located at various latitudes, depending on factors like the intended orbit, launch vehicle performance, and geographical constraints.
2. Are all launch sites in very close proximity to the coast?
Yes, most launch sites appear to be located near the coast. This is likely due to several reasons:

Accessibility: Coastal locations provide easier access for shipping and transporting large rocket components.

Safety: Launching over the ocean reduces the risk of damage and casualties in the event of a launch failure.

Orbital trajectories: Launching near the coast allows for more efficient orbital trajectories, as rockets can take advantage of the Earth's rotation to gain additional velocity.

However, some launch sites, like Plesetsk Cosmodrome (Russia) and Baikonur Cosmodrome (Kazakhstan), are located inland. These sites were established during the Cold War era, and their locations were influenced by strategic and geopolitical considerations.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [82]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [83]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [84]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Create a new column 'marker_color' based on the 'class' value
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']].drop_duplicates()
launch_sites_df['marker_color'] = launch_sites_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [85]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [87]:
# Add marker_cluster to current site_map
"""Build_a_Dashboard_Application_with_Plotly_Dash.pdf
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map
"""
# for each row in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinate
    # and customize the Marker's icon property to indicate if this launch was successful or failed
    marker = folium.Marker([record['Lat'], record['Long']], 
                             icon=folium.Icon(color='white', icon_color='green' if record['class'] == 1 else 'red'))
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [88]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [89]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [90]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Define the launch site coordinates
launch_site_lat = launch_sites_df.loc[0, 'Lat']
launch_site_lon = launch_sites_df.loc[0, 'Long']

# Define the coastline coordinates (obtained using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Print the result
print("Distance to coastline:", distance_coastline, "km")

Distance to coastline: 0.5797581813109574 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [91]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Define the launch site coordinates
launch_site_lat = launch_sites_df.loc[0, 'Lat']
launch_site_lon = launch_sites_df.loc[0, 'Long']

# Define the coastline coordinates (obtained using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium.Marker to display the distance
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(50,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>Distance to Launch Site: {:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [92]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)

# Define the launch site coordinates
launch_site_lat = launch_sites_df.loc[0, 'Lat']
launch_site_lon = launch_sites_df.loc[0, 'Long']

# Define the coastline coordinates (obtained using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Define the coordinates for the PolyLine
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create a PolyLine object
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [93]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Define the launch site coordinates
launch_site_lat = launch_sites_df.loc[0, 'Lat']
launch_site_lon = launch_sites_df.loc[0, 'Long']

# Define the coordinates of the closest city, railway, highway, etc. (obtained using MousePosition)
city_lat = 28.53836
city_lon = -80.60466

# Calculate the distance between the launch site and the closest city, railway, highway, etc.
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a marker with distance to the closest city, railway, highway, etc.
city_marker = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(50,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>Distance to Launch Site: {:10.2f} KM</b></div>'.format(distance_city),
    )
)

# Add the marker to the map
site_map.add_child(city_marker)

# Define the coordinates for the PolyLine

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


Based on the distance lines plotted to the proximities, here are the answers to the questions:

Are launch sites in close proximity to railways?

Observation: Some launch sites have railways nearby, while others do not.
Explanation: Launch sites may be located near railways to facilitate the transportation of heavy rocket components and other equipment. However, not all launch sites require direct access to railways, and some may rely on other modes of transportation.


Are launch sites in close proximity to highways?

Observation: Most launch sites have highways nearby.
Explanation: Launch sites often require good road connectivity to transport personnel, equipment, and rocket components. Highways provide a convenient and efficient way to connect launch sites to nearby cities, airports, and other infrastructure.

Are launch sites in close proximity to coastline?

Observation: Many launch sites are located near the coastline.
Explanation: Launch sites near the coastline can take advantage of the Earth's rotation to gain additional velocity during launches. This is particularly beneficial for launches into equatorial orbits. Additionally, coastal locations can provide a safe and convenient area for rocket stages to fall after separation.

Do launch sites keep certain distance away from cities?

Observation: Launch sites are often located at a significant distance from major cities.
Explanation: Launch sites are typically located in remote areas to minimize the risk of damage and casualties in the event of a launch failure. Additionally, launch sites require a large area for the launch pad, support facilities, and safety zones, which can be difficult to accommodate near populated cities.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.


In [ ]:
#Show a scatter plot of Flight Number vs. Launch Site

import matplotlib.pyplot as plt

# Extract the flight number and launch site data
flight_numbers = spacex_df.index
launch_sites = spacex_df['Launch Site']

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(flight_numbers, launch_sites, c='blue', alpha=0.5)
plt.title('Flight Number vs. Launch Site')
plt.xlabel('Flight Number')
plt.ylabel('Launch Site')
plt.grid(True)
plt.show()

In [ ]:
print(spacex_df.columns)
print(spacex_df.head())

import matplotlib.pyplot as plt

# Extract the payload mass and launch site data
payload_mass = spacex_df['Payload Mass']
launch_sites = spacex_df['Launch Site']

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(launch_sites, payload_mass, c='blue', alpha=0.5)
plt.title('Payload vs. Launch Site')
plt.xlabel('Launch Site')
plt.ylabel('Payload Mass (kg)')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Show a bar chart for the success rate of each orbit type
# Calculate the success rate for each orbit type
orbit_success_rate = spacex_df.groupby('Orbit')['class'].mean()

# Plot the success rate as a bar chart
plt.figure(figsize=(10, 6))
orbit_success_rate.plot(kind='bar', color='skyblue')
plt.title('Success Rate of Each Orbit Type')
plt.xlabel('Orbit Type')
plt.ylabel('Success Rate')
plt.grid(axis='y')
plt.show()


In [ ]:
# Filter the launch_df DataFrame to find rows where launch site starts with 'CCA'
cca_launches = launch_sites_df[launch_sites_df['Launch Site'].str.startswith('CCA')]

# Print the first 5 records
print(cca_launches.head(5))

In [ ]:
spacex_df.columns

In [ ]:
# Print the number of rows where the customer is NASA
print(len(spacex_df[spacex_df['Customer'] == 'NASA']))

# Print the unique values in the 'Customer' column
print(spacex_df['Customer'].unique())

# Print the data types of the columns in the spacex_df DataFrame
print(spacex_df.dtypes)

# Print the first few rows of the spacex_df DataFrame
print(spacex_df.head())

In [ ]:
#Calculate the total payload masses

# Filter the spacex_df DataFrame to find rows where customer contains 'NASA'
nasa_launches = spacex_df[spacex_df['Customer'].str.contains('NASA')]

# Check if the 'Payload Mass (kg)' column contains string values
if nasa_launches['Payload Mass (kg)'].dtype == object:
    # Remove any commas from the payload mass values and convert to float
    total_payload = nasa_launches['Payload Mass (kg)'].str.replace(',', '').astype(float).sum()
else:
    # The column already contains numeric values, so just sum them up
    total_payload = nasa_launches['Payload Mass (kg)'].sum()

# Print the result
print(f'Total payload carried by NASA: {total_payload} kg')

In [ ]:
#Average payload mass
# Filter the spacex_df DataFrame to find rows where booster version is 'F9 v1.1'
f9_v11_launches = spacex_df[spacex_df['Booster Version'] == 'F9 v1.1']

# Check if the 'Payload Mass (kg)' column contains string values
if f9_v11_launches['Payload Mass (kg)'].dtype == object:
    # Remove any commas from the payload mass values and convert to float
    f9_v11_payload_masses = f9_v11_launches['Payload Mass (kg)'].str.replace(',', '').astype(float)
else:
    # The column already contains numeric values
    f9_v11_payload_masses = f9_v11_launches['Payload Mass (kg)']

# Calculate the average payload mass
average_payload_mass = f9_v11_payload_masses.mean()

# Print the result
print(f'Average payload mass carried by F9 v1.1: {average_payload_mass} kg')

In [ ]:
# Filter the spacex_df DataFrame to find rows where landing outcome is 'Success (ground pad)' or 'Success (drone ship)'
successful_landings = spacex_df[(spacex_df['Landing Outcome'] == 'Success (ground pad)') | (spacex_df['Landing Outcome'] == 'Success (drone ship)')]

# Check if the successful_landings DataFrame is not empty
if not successful_landings.empty:
    # Sort the successful landings by date and find the first one
    first_successful_landing = successful_landings.sort_values(by='Date').iloc[0]
    # Print the date of the first successful landing
    print('Date of first successful landing:', first_successful_landing['Date'])
else:
    print('No successful landings found.')

In [ ]:
# Successful Drone Ship Landing with Payload between 4000 and 6000
# Filter the spacex_df DataFrame to find rows where landing outcome is 'Success (drone ship)' or 'Success (ground pad)'
# and payload mass is greater than 4000 but less than 6000
successful_landings = spacex_df[
    ((spacex_df['Landing Outcome'] == 'Success (drone ship)') | (spacex_df['Landing Outcome'] == 'Success (ground pad)')) &
    (spacex_df['Payload Mass (kg)'].astype(float) > 4000) &
    (spacex_df['Payload Mass (kg)'].astype(float) < 6000)
]

# Print the booster names
print(successful_landings[['Flight Number', 'Date', 'Booster Version']])